# Treinamento de rede neural para queda livre (Google Colab)

Notebook pronto para rodar no Google Colab. Gera uma base sintética com a equação da queda livre, treina uma rede neural (Keras) que prediz a posição y dada a altura inicial h0 e o tempo t, salva o dataset e o modelo no Google Drive (opcional) e faz comparação entre predição e fórmula analítica.

Rodar as células na ordem. Comentários em Português.

## 1) Configuração e montagem do Google Drive (opcional)

In [ ]:
# Montar o Google Drive (opcional)
from google.colab import drive
drive.mount('/content/drive')
# Defina uma pasta no Drive para salvar dataset e modelo, por exemplo:
DRIVE_DIR = '/content/drive/MyDrive/ObjetoQueda'
import os
os.makedirs(DRIVE_DIR, exist_ok=True)
print('Pasta de salvamento:', DRIVE_DIR)

## 2) Geração da base de dados (equação da queda livre)

In [ ]:
# Gera dados sintéticos da queda livre e salva em CSV
import numpy as np
import pandas as pd

# Parâmetros físicos
g = 9.80665  # m/s^2

def posicao_queda(h0, t, g=g):
    """Equação analítica: y(t) = h0 - 0.5*g*t^2
    Retorna posição y (m).
    """
    return h0 - 0.5 * g * t**2

# Gerar dataset
np.random.seed(42)
N_heights = 500  # diferentes alturas iniciais
N_times_per_height = 50
h0_vals = np.random.uniform(0.5, 100.0, size=N_heights)  # alturas entre 0.5 e 100 m
data = []
for h0 in h0_vals:
    # tempos até o objeto atingir o chão (resolver h0 - 0.5*g*t^2 = 0)
    t_max = np.sqrt(2 * h0 / g)
    # amostras de tempo uniformes entre 0 e t_max
    ts = np.random.uniform(0, t_max, size=N_times_per_height)
    ys = posicao_queda(h0, ts)
    # adicionar ruído pequeno nas posições para simular medições
    noise = np.random.normal(scale=0.01 * h0, size=ys.shape)  # ruído proporcional
    ys_noisy = ys + noise
    for t, y in zip(ts, ys_noisy):
        data.append([h0, t, y])

df = pd.DataFrame(data, columns=['h0', 't', 'y'])
print('Dataset gerado com', len(df), 'amostras')
# salvar localmente e no Drive
LOCAL_DIR = '/content'
local_csv = os.path.join(LOCAL_DIR, 'objeto_queda_dataset.csv')
df.to_csv(local_csv, index=False)
print('Salvo em', local_csv)
drive_csv = os.path.join(DRIVE_DIR, 'objeto_queda_dataset.csv')
df.to_csv(drive_csv, index=False)
print('Também salvo em Drive:', drive_csv)

## 3) Treinamento da rede neural (Keras)

In [ ]:
# Treinar uma rede neural simples para predizer y a partir de h0 e t
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# carregar dataset
df = pd.read_csv(local_csv)
# features e alvo
X = df[['h0', 't']].values
y = df['y'].values

# dividir em treino/val/teste
from sklearn.model_selection import train_test_split
X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.5, random_state=42)

# normalização simples
from sklearn.preprocessing import StandardScaler
scaler_X = StandardScaler().fit(X_train)
X_train_s = scaler_X.transform(X_train)
X_val_s = scaler_X.transform(X_val)
X_test_s = scaler_X.transform(X_test)

# modelo
def build_model():
    model = keras.Sequential([
        layers.Input(shape=(2,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='linear')
    ])
    model.compile(optimizer=keras.optimizers.Adam(1e-3), loss='mse', metrics=['mae'])
    return model

model = build_model()
model.summary()

# callbacks para salvar melhor peso
checkpoint_path = os.path.join(DRIVE_DIR, 'model_checkpoint.h5')
cp = keras.callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True, monitor='val_loss')
es = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train_s, y_train, epochs=200, batch_size=64, validation_data=(X_val_s, y_val), callbacks=[cp, es])

# salvar o modelo final
final_model_path = os.path.join(DRIVE_DIR, 'modelo_objeto_queda.h5')
model.save(final_model_path)
print('Modelo salvo em', final_model_path)

## 4) Avaliação e comparação com a fórmula analítica

In [ ]:
# Carregar modelo salvo e comparar previsões com fórmula analítica
from tensorflow.keras.models import load_model
model = load_model(final_model_path)

# previsões no conjunto de teste
y_pred = model.predict(X_test_s).flatten()
# comparação com fórmula analítica sem ruído
h0_test = X_test[:,0]
t_test = X_test[:,1]
y_true_analytic = posicao_queda(h0_test, t_test)

from sklearn.metrics import mean_squared_error, mean_absolute_error
mse_model = mean_squared_error(y_test, y_pred)
mae_model = mean_absolute_error(y_test, y_pred)
mse_analytic = mean_squared_error(y_test, y_true_analytic)
mae_analytic = mean_absolute_error(y_test, y_true_analytic)

print('MSE (modelo) =', mse_model)
print('MAE (modelo) =', mae_model)
print('MSE (analítico) =', mse_analytic)
print('MAE (analítico) =', mae_analytic)

# plots
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
plt.scatter(y_test, y_pred, s=5, alpha=0.6, label='modelo pred vs medidos')
plt.scatter(y_test, y_true_analytic, s=5, alpha=0.6, label='analítico vs medidos')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--')
plt.legend()
plt.xlabel('y medido (com ruído)')
plt.ylabel('y previsto')
plt.title('Comparação: modelo ML vs fórmula analítica')
plt.show()

# salvar resultados de comparação
results_df = pd.DataFrame({'h0':h0_test, 't':t_test, 'y_measured': y_test, 'y_pred': y_pred, 'y_analytic': y_true_analytic})
results_csv = os.path.join(DRIVE_DIR, 'comparacao_resultados.csv')
results_df.to_csv(results_csv, index=False)
print('Resultados salvos em', results_csv)